# CS 5588 — Week 1 Hands On
## From Data to Retrieval: GitHub → Colab → Hugging Face → Embeddings

**Learning Goals:**
- Use GitHub for collaborative analytics workflows
- Run notebooks in Google Colab
- Load datasets and models from Hugging Face Hub
- Build an embedding-based retrieval system (mini-RAG)


### GenAI Systems Context (Mini-RAG)
This lab implements a **mini Retrieval-Augmented Generation (RAG)** pipeline:
- A **Transformer encoder** produces semantic embeddings
- A **vector index (FAISS)** enables fast retrieval
- Retrieved context is what a downstream **LLM** would use for grounded generation


## Step 1 — Environment Setup
Install required libraries. This may take ~1 minute.


In [1]:
!pip install -q transformers datasets sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 50.8 MB/s eta 0:00:00


## Step 2 — Load Dataset & Model from Hugging Face Hub
We use a lightweight news dataset and a sentence embedding model.


Replace you hugging face token in the empty string, on the mentioned commented line.

In [3]:
from huggingface_hub import login

HF_TOKEN = ""  # <-- REPLACE THE EMPTY STRING WITH YOUR HF TOKEN

if HF_TOKEN and HF_TOKEN != "":
    login(token=HF_TOKEN)
    print("✅ Logged in to Hugging Face")
else:
    print("⚠️ No HF token provided. Public models may still work, but rate limits may apply.")

⚠️ No HF token provided. Public models may still work, but rate limits may apply.


TASK

1. Find out any /ag_news dataset from the huggingface.
2. Look for "Use this dataset" button on the left side --> Use huggingface library option.
3. Copy the entire code and paste in the empty cell and run it successfully.

In [4]:
#paste your dataset code from huggingface here
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

dataset = load_dataset("ag_news", split="train[:200]")
model = SentenceTransformer("all-MiniLM-L6-v2")

texts = dataset["text"]
embeddings = model.encode(texts, show_progress_bar=True)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
# dataset

In [8]:
# texts = dataset["train"].select(range(200))
# print(f"Selected {len(texts)} examples from the 'train' split.")

In [9]:
texts[1:6]

['Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.',
 "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.",
 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.',
 'Oil prices soar to all-time record, posing new menace to US economy (AFP) AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US president

TASK

1. Find out the sentence-transformer -
all-MiniLM-L6-v2 from the huggingface website.
2. Look for "Use this model" button on the left side --> Use sentence-transformer library option.
3. Copy the first 2 lines of the code and paste in the empty cell and run it successfully.

In [ ]:
#paste your first 2 lines of the sentence-transformer library code from the hugginface to load the model


## Step 3 — Create Embeddings
These vectors represent semantic meaning and enable retrieval before generation.


In [10]:
# embeddings = model.encode(texts, show_progress_bar=True)
print('Embedding shape:', embeddings.shape)

Embedding shape: (200, 384)


## Step 4 — Build a Vector Index (FAISS)
This simulates the retrieval layer in RAG systems.


In [11]:
import faiss
import numpy as np

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))
print('Index size:', index.ntotal)

Index size: 200


## Step 5 — Retrieval Function
Search for documents related to a query.


In [12]:
def search(query, k=3):
    q_emb = model.encode([query])
    distances, indices = index.search(np.array(q_emb), k)
    return [texts[int(i)] for i in indices[0]]

## Step 6 — Try It!


In [15]:
query = "no intelligence in healthcare"
top_chunks = search(query, k=3)

print(f"Top 3 chunks retrieved for query: '{query}'\n")
for i, chunk in enumerate(top_chunks):
    print(f"{i+1}. {chunk}\n")

Top 3 chunks retrieved for query: 'no intelligence in healthcare'

1. U.K.'s NHS taps Gartner to help plan \$9B IT overhaul LONDON -- The U.K.'s National Health Service (NHS) has tapped IT researcher Gartner Inc. to provide market intelligence services as the health organization forges ahead with a mammoth, 5 billion (\$9.2 billion) project to upgrade its information technology infrastructure.

2. HP: The Adaptive Enterprise that can't adapt &lt;strong&gt;Opinion&lt;/strong&gt; SAP hardly to blame

3. Steady as they go BEDFORD -- Scientists at NitroMed Inc. hope their experimental drugs will cure heart disease someday. But lately their focus has been on more mundane matters.



In [16]:
search("stock market and economy")

["Oil and Economy Cloud Stocks' Outlook  NEW YORK (Reuters) - Soaring crude prices plus worries  about the economy and the outlook for earnings are expected to  hang over the stock market next week during the depth of the  summer doldrums.",
 "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.",
 'Stocks End Up, But Near Year Lows (Reuters) Reuters - Stocks ended slightly higher on Friday\\but stayed near lows for the year as oil prices surged past  #36;46\\a barrel, offsetting a positive outlook from computer maker\\Dell Inc. (DELL.O)']

In [21]:
for x in search("artificial intelligence in healthcare"):
    print(x)

U.K.'s NHS taps Gartner to help plan \$9B IT overhaul LONDON -- The U.K.'s National Health Service (NHS) has tapped IT researcher Gartner Inc. to provide market intelligence services as the health organization forges ahead with a mammoth, 5 billion (\$9.2 billion) project to upgrade its information technology infrastructure.
UK Scientists Allowed to Clone Human Embryos (Reuters) Reuters - British scientists said on Wednesday\they had received permission to clone human embryos for medical\research, in what they believe to be the first such license to\be granted in Europe.
Coming to The Rescue Got a unique problem? Not to worry: you can find a financial planner for every specialized need


## Reflection
In 1–2 sentences, explain how embeddings enable retrieval before generation in GenAI systems.


Embeddings map texts and queries into a shared vector space where semantic similarity corresponds to geometric proximity. Retrieval runs as a nearest-neighbor search over these vectors (e.g., FAISS) to select the most relevant chunks to pass as context for downstream generation.